In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated,Literal,Any ,Dict
from langchain_core.messages import BaseMessage,HumanMessage,SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
import operator
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("Groq api key not found in environment")

llm = ChatOpenAI(
    model='openai/gpt-oss-120b',
    temperature=0,
    max_tokens = 8192,
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

In [3]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explanation:str

In [4]:
def generate_joke(state:JokeState) -> Dict[str,Any]:
    topic = state["topic"]
    prompt = f"Generate a joke on a topic : {topic}"

    response = llm.invoke(prompt)
    return {'joke':response}

def generate_explanation(state:JokeState) -> Dict[str,Any]:
    joke = state["joke"]
    prompt = f"Write an explanation for the joke : {joke}"

    response = llm.invoke(prompt)

    return {"explanation":response}

In [5]:
graph = StateGraph(JokeState)
graph.add_node("generate_joke",generate_joke)
graph.add_node("generate_explanation",generate_explanation)

graph.add_edge(START,"generate_joke")
graph.add_edge("generate_joke","generate_explanation")
graph.add_edge("generate_explanation",END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [6]:
config1 = {"configurable":{"thread_id":"1"}}
workflow.invoke({"topic":"pizza"},config=config1)

{'topic': 'pizza',
 'joke': AIMessage(content='Why did the pizza apply for a job?\n\nBecause it heard the company was looking for someone with *dough*‑mestic skills and a *crust*‑worthy work ethic! 🍕😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 79, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 20, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.054782574, 'prompt_time': 0.003580746, 'completion_time': 0.152725876, 'total_time': 0.156306622}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'id': 'chatcmpl-b4c15b6b-9236-4c11-9876-a79948965078', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3205-71a0-be00-e7e1ec82f12e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 71, 'total_tokens': 150, 

In [7]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': AIMessage(content='Why did the pizza apply for a job?\n\nBecause it heard the company was looking for someone with *dough*‑mestic skills and a *crust*‑worthy work ethic! 🍕😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 79, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 20, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.054782574, 'prompt_time': 0.003580746, 'completion_time': 0.152725876, 'total_time': 0.156306622}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'id': 'chatcmpl-b4c15b6b-9236-4c11-9876-a79948965078', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3205-71a0-be00-e7e1ec82f12e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 71, '

In [8]:
list(workflow.get_state_history(config1)) # use to see at each step what happen in all nodes 

[StateSnapshot(values={'topic': 'pizza', 'joke': AIMessage(content='Why did the pizza apply for a job?\n\nBecause it heard the company was looking for someone with *dough*‑mestic skills and a *crust*‑worthy work ethic! 🍕😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 79, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 20, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.054782574, 'prompt_time': 0.003580746, 'completion_time': 0.152725876, 'total_time': 0.156306622}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'id': 'chatcmpl-b4c15b6b-9236-4c11-9876-a79948965078', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3205-71a0-be00-e7e1ec82f12e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 71, 

In [9]:
config2 = {"configurable":{"thread_id":"2"}}
workflow.invoke({"topic":"pasta"},config=config2)

{'topic': 'pasta',
 'joke': AIMessage(content='Why did the spaghetti get a promotion?\n\nBecause it knew how to *pasta* the competition and always *rolled* with the punches! 🍝😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 79, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 15, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.086199587, 'prompt_time': 0.004859283, 'completion_time': 0.118414715, 'total_time': 0.123273998}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'id': 'chatcmpl-db734a4d-87c0-4670-8352-124ccde2b9f5', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3e7e-7030-8419-253777ed88af-0', usage_metadata={'input_tokens': 79, 'output_tokens': 56, 'total_tokens': 135, 'input_token_details': {

In [ ]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': AIMessage(content='Why did the spaghetti get a promotion?\n\nBecause it knew how to *pasta* the competition and always *rolled* with the punches! 🍝😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 79, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 15, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.086199587, 'prompt_time': 0.004859283, 'completion_time': 0.118414715, 'total_time': 0.123273998}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'id': 'chatcmpl-db734a4d-87c0-4670-8352-124ccde2b9f5', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3e7e-7030-8419-253777ed88af-0', usage_metadata={'input_tokens': 79, 'output_tokens': 56, 'total_tokens': 135, 'in

## Persistence

               Persistence
            ┌───────┼────────┬──────────┐
            │       │        │          │
     Short-term     HITL   Fault       Time
        Memory            Tolerance    Travel


# Fault Tolerance

In [11]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [12]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [13]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [14]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

In [ ]:
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

## TimeTravel

In [18]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0e4f22-ea17-6fde-8002-7e0657d33bef"}})

StateSnapshot(values={'topic': 'pizza', 'joke': AIMessage(content='Why did the pizza apply for a job?\n\nBecause it heard the company was looking for someone with *dough*‑mestic skills and a *crust*‑worthy work ethic! 🍕😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 79, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 20, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.054782574, 'prompt_time': 0.003580746, 'completion_time': 0.152725876, 'total_time': 0.156306622}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'id': 'chatcmpl-b4c15b6b-9236-4c11-9876-a79948965078', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3205-71a0-be00-e7e1ec82f12e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 71, '

In [19]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0e4f22-ea17-6fde-8002-7e0657d33bef"}})

{'topic': 'pizza',
 'joke': AIMessage(content='Why did the pizza apply for a job?\n\nBecause it heard the company was looking for someone with *dough*‑mestic skills and a *crust*‑worthy work ethic! 🍕😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 79, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 20, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.054782574, 'prompt_time': 0.003580746, 'completion_time': 0.152725876, 'total_time': 0.156306622}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'id': 'chatcmpl-b4c15b6b-9236-4c11-9876-a79948965078', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3205-71a0-be00-e7e1ec82f12e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 71, 'total_tokens': 150, 

In [20]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': AIMessage(content='Why did the pizza apply for a job?\n\nBecause it heard the company was looking for someone with *dough*‑mestic skills and a *crust*‑worthy work ethic! 🍕😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 79, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 20, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.054782574, 'prompt_time': 0.003580746, 'completion_time': 0.152725876, 'total_time': 0.156306622}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'id': 'chatcmpl-b4c15b6b-9236-4c11-9876-a79948965078', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b6bba-3205-71a0-be00-e7e1ec82f12e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 71, 